In [385]:
class conv2d:
    def __init__(self, k, s, p):
        self.k = k
        self.p = p
        self.s = s

    def get_x(self):
        return self.X + self.p * 2

    def get_y(self):
        return self.Y + self.p * 2

    def get_slides_done(self):# Find slides that are done already
        cols_done = self.y_id * self.y_size + self.p
        h_slides_done = max(math.floor((cols_done-self.k)/self.s) + 1, 0)
        rows_done = self.x_id * self.x_size + self.p
        v_slides_done = max(math.floor((rows_done-self.k)/self.s) + 1, 0)

        # Windows done so far
        return (v_slides_done, h_slides_done)

    def get_block_win_val(self, w, i, j, ch, imgs, img):
        cols_done = self.y_id * self.y_size + self.p
        rows_done = self.x_id * self.x_size + self.p

        h_slides = math.floor((self.get_y()-self.k)/self.s) + 1
        v_slides = math.floor((self.get_x()-self.k)/self.s) + 1

        r = math.floor(w / h_slides)
        c = w % h_slides
        x = (r * self.s) + i
        y = (c * self.s) + j

        print("ORIGIN: ", r * self.s ,c * self.s , "COORD:", x,y, "TCOORD: ", x-rows_done, y-cols_done)
        x -= rows_done
        y -= cols_done

        if x < 0 or y < 0 or x >= self.b_X or y >= self.b_Y:
            return 0

        return imgs[img][ch][x][y]

    def img_block_to_conv2d(self, imgs, X, Y, x_id, y_id, x_size, y_size):
        self.channels, self.b_X, self.b_Y = imgs[0].shape
        self.X, self.Y, self.x_id, self.y_id, self.x_size, self.y_size = X, Y, x_id, y_id, x_size, y_size
        
        cols_done = self.y_id * self.y_size
        rows_done = self.x_id * self.x_size

        h_slides = math.floor((self.get_y()-self.k)/self.s) + 1
        v_slides = math.floor((self.get_x()-self.k)/self.s) + 1

        v_cols = cols_done + self.b_Y + self.p
        v_rows = rows_done + self.b_X + self.p
        if cols_done + self.b_Y >= Y:
            v_cols += self.p
        if rows_done + self.b_X >= X:
            v_rows += self.p

        h_vslides = max(math.floor((v_cols - self.k)/self.s) + 1, 0)
        v_vslides = max(math.floor((v_rows - self.k)/self.s) + 1, 0)
        
        if cols_done + self.b_Y < Y:
            h_vslides += 1
        if rows_done + self.b_X < X:
            v_vslides += 1
        
        print("Virtual matrix slides ", h_vslides, v_vslides)

        v_done, h_done = self.get_slides_done()
        print("Starting window ", v_done, h_done)

        data = {}
        for x in range(v_done, v_vslides):
            for y in range(h_done, h_vslides):
                print("Looking at window ", x * h_slides + y)
                row = []
                for c in range(self.channels):
                    for i in range(self.k):
                        for j in range(self.k):
                            row.append(self.get_block_win_val(x * h_slides + y, i, j, c, imgs, 0))
                data[x * h_slides+ y] = np.array(row)

        return data
 
    def aggregate(self, datas):
        fin = {}
        for partial in datas:
            for window in partial:
                if window in fin:
#                     for i in range(len(fin[window])):
#                         if fin[window][i] == 0:
#                             fin[window][i] = partial[window][i]
#                         else:
#                             assert fin[window][i] == partial[window][i]
                    fin[window] += partial[window]
                else:
                    fin[window] = partial[window]
        res = [fin[d] for d in range(len(fin))]
        return np.array(res)

    def get_win_val(self, w, i, j, ch, imgs, img):    
        h_slides = math.floor((self.get_y()-self.k)/self.s) + 1
        v_slides = math.floor((self.get_x()-self.k)/self.s) + 1

        r = math.floor(w / h_slides)
        c = w % h_slides
        x = (r * self.s) + i - self.p
        y = (c * self.s) + j - self.p

        print("ORIGIN: ", r * self.s ,c * self.s , "COORD:", x,y)
        if x < 0 or y < 0 or x >= self.X or y >= self.Y:
            return 0

        return imgs[img][ch][x][y]
    
    def img_to_conv2d(self, imgs):
        self.channels, self.X, self.Y = imgs[0].shape

        h_slides = math.floor((self.get_y()-self.k)/self.s) + 1
        v_slides = math.floor((self.get_x()-self.k)/self.s) + 1

        windows = h_slides * v_slides

        data = []

        for w in range(windows):
            print("Looking at window ", w)
            row = []
            for c in range(self.channels):
                for i in range(self.k):
                    for j in range(self.k):
                        row.append(self.get_win_val(w, i, j, c, imgs, 0))
            data.append(row)

        return np.array(data)

In [324]:
imgs = np.random.rand(1,1,10,10)

kernel_size = 3
stride = 2
padding = 0

con = conv2d(kernel_size, stride, padding)
data = con.img_to_conv2d(imgs)
print(data.shape)
for i in range(data.shape[0]):
    print(data[i].tolist())

(16, 9)
[0.5985788753408876, 0.791789814752671, 0.562428837871263, 0.44813312403788175, 0.42915768861666037, 0.7371383498803802, 0.32587704092010605, 0.9294371685291173, 0.5805748874095275]
[0.562428837871263, 0.5079950764209281, 0.36832765822349645, 0.7371383498803802, 0.27734705233035206, 0.6046147528210861, 0.5805748874095275, 0.7615311560756912, 0.24565528816737192]
[0.36832765822349645, 0.858656146412357, 0.191250568460818, 0.6046147528210861, 0.43507917200736046, 0.6021307989694207, 0.24565528816737192, 0.6168534588703357, 0.30086762321039373]
[0.191250568460818, 0.9309487536653095, 0.7815012427329986, 0.6021307989694207, 0.9658956243270637, 0.042347136648001604, 0.30086762321039373, 0.026070859584771733, 0.4686337059989968]
[0.32587704092010605, 0.9294371685291173, 0.5805748874095275, 0.8976484314962087, 0.9345290038414207, 0.28093304749462866, 0.532235511549739, 0.5066627707243655, 0.888218733881415]
[0.5805748874095275, 0.7615311560756912, 0.24565528816737192, 0.28093304749462

In [325]:
for i in range(imgs[0][0].shape[0]):
    print(imgs[0][0][i].tolist())

[0.5985788753408876, 0.791789814752671, 0.562428837871263, 0.5079950764209281, 0.36832765822349645, 0.858656146412357, 0.191250568460818, 0.9309487536653095, 0.7815012427329986, 0.8669279017390918]
[0.44813312403788175, 0.42915768861666037, 0.7371383498803802, 0.27734705233035206, 0.6046147528210861, 0.43507917200736046, 0.6021307989694207, 0.9658956243270637, 0.042347136648001604, 0.14600562484860702]
[0.32587704092010605, 0.9294371685291173, 0.5805748874095275, 0.7615311560756912, 0.24565528816737192, 0.6168534588703357, 0.30086762321039373, 0.026070859584771733, 0.4686337059989968, 0.007205327188668287]
[0.8976484314962087, 0.9345290038414207, 0.28093304749462866, 0.5589314883275227, 0.9316332997466353, 0.4209860013704235, 0.04790161929530956, 0.8079065084340876, 0.8705509245898838, 0.19552457694351266]
[0.532235511549739, 0.5066627707243655, 0.888218733881415, 0.11291301052131031, 0.6499179818705164, 0.8455480951961718, 0.9933248661835061, 0.8529608610836009, 0.4389572952651447, 0.

In [326]:
b_0_0 = np.array(imgs[0][0][:5,:5]).reshape(1,1,5,5)
b_0_1 = np.array(imgs[0][0][:5,5:]).reshape(1,1,5,5)
b_1_0 = np.array(imgs[0][0][5:,:5]).reshape(1,1,5,5)
b_1_1 = np.array(imgs[0][0][5:,5:]).reshape(1,1,5,5)

In [327]:
con = conv2d(kernel_size, stride, padding)
data1 = con.img_block_to_conv2d(b_0_0, 10, 10, 0, 0, 5, 5)
data1
# print(data1.shape)
# for i in range(data1.shape[0]):
#     print(data1[i].tolist())

Virtual matrix slides  3 3
Starting window  0 0
Looking at window  0
ORIGIN:  0 0 COORD: 0 0 TCOORD:  0 0
ORIGIN:  0 0 COORD: 0 1 TCOORD:  0 1
ORIGIN:  0 0 COORD: 0 2 TCOORD:  0 2
ORIGIN:  0 0 COORD: 1 0 TCOORD:  1 0
ORIGIN:  0 0 COORD: 1 1 TCOORD:  1 1
ORIGIN:  0 0 COORD: 1 2 TCOORD:  1 2
ORIGIN:  0 0 COORD: 2 0 TCOORD:  2 0
ORIGIN:  0 0 COORD: 2 1 TCOORD:  2 1
ORIGIN:  0 0 COORD: 2 2 TCOORD:  2 2
Looking at window  1
ORIGIN:  0 2 COORD: 0 2 TCOORD:  0 2
ORIGIN:  0 2 COORD: 0 3 TCOORD:  0 3
ORIGIN:  0 2 COORD: 0 4 TCOORD:  0 4
ORIGIN:  0 2 COORD: 1 2 TCOORD:  1 2
ORIGIN:  0 2 COORD: 1 3 TCOORD:  1 3
ORIGIN:  0 2 COORD: 1 4 TCOORD:  1 4
ORIGIN:  0 2 COORD: 2 2 TCOORD:  2 2
ORIGIN:  0 2 COORD: 2 3 TCOORD:  2 3
ORIGIN:  0 2 COORD: 2 4 TCOORD:  2 4
Looking at window  2
ORIGIN:  0 4 COORD: 0 4 TCOORD:  0 4
ORIGIN:  0 4 COORD: 0 5 TCOORD:  0 5
ORIGIN:  0 4 COORD: 0 6 TCOORD:  0 6
ORIGIN:  0 4 COORD: 1 4 TCOORD:  1 4
ORIGIN:  0 4 COORD: 1 5 TCOORD:  1 5
ORIGIN:  0 4 COORD: 1 6 TCOORD:  1 6
O

{0: array([0.59857888, 0.79178981, 0.56242884, 0.44813312, 0.42915769,
        0.73713835, 0.32587704, 0.92943717, 0.58057489]),
 1: array([0.56242884, 0.50799508, 0.36832766, 0.73713835, 0.27734705,
        0.60461475, 0.58057489, 0.76153116, 0.24565529]),
 2: array([0.36832766, 0.        , 0.        , 0.60461475, 0.        ,
        0.        , 0.24565529, 0.        , 0.        ]),
 4: array([0.32587704, 0.92943717, 0.58057489, 0.89764843, 0.934529  ,
        0.28093305, 0.53223551, 0.50666277, 0.88821873]),
 5: array([0.58057489, 0.76153116, 0.24565529, 0.28093305, 0.55893149,
        0.9316333 , 0.88821873, 0.11291301, 0.64991798]),
 6: array([0.24565529, 0.        , 0.        , 0.9316333 , 0.        ,
        0.        , 0.64991798, 0.        , 0.        ]),
 8: array([0.53223551, 0.50666277, 0.88821873, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ]),
 9: array([0.88821873, 0.11291301, 0.64991798, 0.        , 0.        ,
        0.        , 0.    

In [328]:
con = conv2d(kernel_size, stride, padding)
data2 = con.img_block_to_conv2d(b_0_1, 10, 10, 0, 1, 5, 5)
data2
# print(data1.shape)
# for i in range(data1.shape[0]):
#     print(data1[i].tolist())

Virtual matrix slides  4 3
Starting window  0 2
Looking at window  2
ORIGIN:  0 4 COORD: 0 4 TCOORD:  0 -1
ORIGIN:  0 4 COORD: 0 5 TCOORD:  0 0
ORIGIN:  0 4 COORD: 0 6 TCOORD:  0 1
ORIGIN:  0 4 COORD: 1 4 TCOORD:  1 -1
ORIGIN:  0 4 COORD: 1 5 TCOORD:  1 0
ORIGIN:  0 4 COORD: 1 6 TCOORD:  1 1
ORIGIN:  0 4 COORD: 2 4 TCOORD:  2 -1
ORIGIN:  0 4 COORD: 2 5 TCOORD:  2 0
ORIGIN:  0 4 COORD: 2 6 TCOORD:  2 1
Looking at window  3
ORIGIN:  0 6 COORD: 0 6 TCOORD:  0 1
ORIGIN:  0 6 COORD: 0 7 TCOORD:  0 2
ORIGIN:  0 6 COORD: 0 8 TCOORD:  0 3
ORIGIN:  0 6 COORD: 1 6 TCOORD:  1 1
ORIGIN:  0 6 COORD: 1 7 TCOORD:  1 2
ORIGIN:  0 6 COORD: 1 8 TCOORD:  1 3
ORIGIN:  0 6 COORD: 2 6 TCOORD:  2 1
ORIGIN:  0 6 COORD: 2 7 TCOORD:  2 2
ORIGIN:  0 6 COORD: 2 8 TCOORD:  2 3
Looking at window  6
ORIGIN:  2 4 COORD: 2 4 TCOORD:  2 -1
ORIGIN:  2 4 COORD: 2 5 TCOORD:  2 0
ORIGIN:  2 4 COORD: 2 6 TCOORD:  2 1
ORIGIN:  2 4 COORD: 3 4 TCOORD:  3 -1
ORIGIN:  2 4 COORD: 3 5 TCOORD:  3 0
ORIGIN:  2 4 COORD: 3 6 TCOORD:  

{2: array([0.        , 0.85865615, 0.19125057, 0.        , 0.43507917,
        0.6021308 , 0.        , 0.61685346, 0.30086762]),
 3: array([0.19125057, 0.93094875, 0.78150124, 0.6021308 , 0.96589562,
        0.04234714, 0.30086762, 0.02607086, 0.46863371]),
 6: array([0.        , 0.61685346, 0.30086762, 0.        , 0.420986  ,
        0.04790162, 0.        , 0.8455481 , 0.99332487]),
 7: array([0.30086762, 0.02607086, 0.46863371, 0.04790162, 0.80790651,
        0.87055092, 0.99332487, 0.85296086, 0.4389573 ]),
 10: array([0.        , 0.8455481 , 0.99332487, 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ]),
 11: array([0.99332487, 0.85296086, 0.4389573 , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        ])}

In [329]:
con = conv2d(kernel_size, stride, padding)
data3 = con.img_block_to_conv2d(b_1_0, 10, 10, 1, 0, 5, 5)
data3
# print(data1.shape)
# for i in range(data1.shape[0]):
#     print(data1[i].tolist())

Virtual matrix slides  3 4
Starting window  2 0
Looking at window  8
ORIGIN:  4 0 COORD: 4 0 TCOORD:  -1 0
ORIGIN:  4 0 COORD: 4 1 TCOORD:  -1 1
ORIGIN:  4 0 COORD: 4 2 TCOORD:  -1 2
ORIGIN:  4 0 COORD: 5 0 TCOORD:  0 0
ORIGIN:  4 0 COORD: 5 1 TCOORD:  0 1
ORIGIN:  4 0 COORD: 5 2 TCOORD:  0 2
ORIGIN:  4 0 COORD: 6 0 TCOORD:  1 0
ORIGIN:  4 0 COORD: 6 1 TCOORD:  1 1
ORIGIN:  4 0 COORD: 6 2 TCOORD:  1 2
Looking at window  9
ORIGIN:  4 2 COORD: 4 2 TCOORD:  -1 2
ORIGIN:  4 2 COORD: 4 3 TCOORD:  -1 3
ORIGIN:  4 2 COORD: 4 4 TCOORD:  -1 4
ORIGIN:  4 2 COORD: 5 2 TCOORD:  0 2
ORIGIN:  4 2 COORD: 5 3 TCOORD:  0 3
ORIGIN:  4 2 COORD: 5 4 TCOORD:  0 4
ORIGIN:  4 2 COORD: 6 2 TCOORD:  1 2
ORIGIN:  4 2 COORD: 6 3 TCOORD:  1 3
ORIGIN:  4 2 COORD: 6 4 TCOORD:  1 4
Looking at window  10
ORIGIN:  4 4 COORD: 4 4 TCOORD:  -1 4
ORIGIN:  4 4 COORD: 4 5 TCOORD:  -1 5
ORIGIN:  4 4 COORD: 4 6 TCOORD:  -1 6
ORIGIN:  4 4 COORD: 5 4 TCOORD:  0 4
ORIGIN:  4 4 COORD: 5 5 TCOORD:  0 5
ORIGIN:  4 4 COORD: 5 6 TCOO

{8: array([0.        , 0.        , 0.        , 0.83813095, 0.54024454,
        0.89899405, 0.98118695, 0.8558924 , 0.1135949 ]),
 9: array([0.        , 0.        , 0.        , 0.89899405, 0.27092952,
        0.27100048, 0.1135949 , 0.14107572, 0.3353842 ]),
 10: array([0.        , 0.        , 0.        , 0.27100048, 0.        ,
        0.        , 0.3353842 , 0.        , 0.        ]),
 12: array([0.98118695, 0.8558924 , 0.1135949 , 0.39260332, 0.03249181,
        0.14316001, 0.64224336, 0.23909871, 0.94030427]),
 13: array([0.1135949 , 0.14107572, 0.3353842 , 0.14316001, 0.6484594 ,
        0.43967147, 0.94030427, 0.4612056 , 0.42668768]),
 14: array([0.3353842 , 0.        , 0.        , 0.43967147, 0.        ,
        0.        , 0.42668768, 0.        , 0.        ])}

In [330]:
con = conv2d(kernel_size, stride, padding)
data4 = con.img_block_to_conv2d(b_1_1, 10, 10, 1, 1, 5, 5)
data4
# print(data1.shape)
# for i in range(data1.shape[0]):
#     print(data1[i].tolist())

Virtual matrix slides  4 4
Starting window  2 2
Looking at window  10
ORIGIN:  4 4 COORD: 4 4 TCOORD:  -1 -1
ORIGIN:  4 4 COORD: 4 5 TCOORD:  -1 0
ORIGIN:  4 4 COORD: 4 6 TCOORD:  -1 1
ORIGIN:  4 4 COORD: 5 4 TCOORD:  0 -1
ORIGIN:  4 4 COORD: 5 5 TCOORD:  0 0
ORIGIN:  4 4 COORD: 5 6 TCOORD:  0 1
ORIGIN:  4 4 COORD: 6 4 TCOORD:  1 -1
ORIGIN:  4 4 COORD: 6 5 TCOORD:  1 0
ORIGIN:  4 4 COORD: 6 6 TCOORD:  1 1
Looking at window  11
ORIGIN:  4 6 COORD: 4 6 TCOORD:  -1 1
ORIGIN:  4 6 COORD: 4 7 TCOORD:  -1 2
ORIGIN:  4 6 COORD: 4 8 TCOORD:  -1 3
ORIGIN:  4 6 COORD: 5 6 TCOORD:  0 1
ORIGIN:  4 6 COORD: 5 7 TCOORD:  0 2
ORIGIN:  4 6 COORD: 5 8 TCOORD:  0 3
ORIGIN:  4 6 COORD: 6 6 TCOORD:  1 1
ORIGIN:  4 6 COORD: 6 7 TCOORD:  1 2
ORIGIN:  4 6 COORD: 6 8 TCOORD:  1 3
Looking at window  14
ORIGIN:  6 4 COORD: 6 4 TCOORD:  1 -1
ORIGIN:  6 4 COORD: 6 5 TCOORD:  1 0
ORIGIN:  6 4 COORD: 6 6 TCOORD:  1 1
ORIGIN:  6 4 COORD: 7 4 TCOORD:  2 -1
ORIGIN:  6 4 COORD: 7 5 TCOORD:  2 0
ORIGIN:  6 4 COORD: 7 6 

{10: array([0.        , 0.        , 0.        , 0.        , 0.94766201,
        0.52065377, 0.        , 0.61375099, 0.91449704]),
 11: array([0.        , 0.        , 0.        , 0.52065377, 0.29947317,
        0.15974459, 0.91449704, 0.74485123, 0.26378937]),
 14: array([0.        , 0.61375099, 0.91449704, 0.        , 0.83378375,
        0.40896919, 0.        , 0.31788236, 0.7591134 ]),
 15: array([0.91449704, 0.74485123, 0.26378937, 0.40896919, 0.29239932,
        0.06279354, 0.7591134 , 0.74947858, 0.3800546 ])}

In [331]:
for i in range(data.shape[0]):
    print(data[i].tolist())

[0.5985788753408876, 0.791789814752671, 0.562428837871263, 0.44813312403788175, 0.42915768861666037, 0.7371383498803802, 0.32587704092010605, 0.9294371685291173, 0.5805748874095275]
[0.562428837871263, 0.5079950764209281, 0.36832765822349645, 0.7371383498803802, 0.27734705233035206, 0.6046147528210861, 0.5805748874095275, 0.7615311560756912, 0.24565528816737192]
[0.36832765822349645, 0.858656146412357, 0.191250568460818, 0.6046147528210861, 0.43507917200736046, 0.6021307989694207, 0.24565528816737192, 0.6168534588703357, 0.30086762321039373]
[0.191250568460818, 0.9309487536653095, 0.7815012427329986, 0.6021307989694207, 0.9658956243270637, 0.042347136648001604, 0.30086762321039373, 0.026070859584771733, 0.4686337059989968]
[0.32587704092010605, 0.9294371685291173, 0.5805748874095275, 0.8976484314962087, 0.9345290038414207, 0.28093304749462866, 0.532235511549739, 0.5066627707243655, 0.888218733881415]
[0.5805748874095275, 0.7615311560756912, 0.24565528816737192, 0.28093304749462866, 0.5

In [332]:
d = con.aggregate([data1, data2, data3, data4])
for i in range(d.shape[0]):
    print(d[i].tolist())

[0.5985788753408876, 0.791789814752671, 0.562428837871263, 0.44813312403788175, 0.42915768861666037, 0.7371383498803802, 0.32587704092010605, 0.9294371685291173, 0.5805748874095275]
[0.562428837871263, 0.5079950764209281, 0.36832765822349645, 0.7371383498803802, 0.27734705233035206, 0.6046147528210861, 0.5805748874095275, 0.7615311560756912, 0.24565528816737192]
[0.36832765822349645, 0.858656146412357, 0.191250568460818, 0.6046147528210861, 0.43507917200736046, 0.6021307989694207, 0.24565528816737192, 0.6168534588703357, 0.30086762321039373]
[0.191250568460818, 0.9309487536653095, 0.7815012427329986, 0.6021307989694207, 0.9658956243270637, 0.042347136648001604, 0.30086762321039373, 0.026070859584771733, 0.4686337059989968]
[0.32587704092010605, 0.9294371685291173, 0.5805748874095275, 0.8976484314962087, 0.9345290038414207, 0.28093304749462866, 0.532235511549739, 0.5066627707243655, 0.888218733881415]
[0.5805748874095275, 0.7615311560756912, 0.24565528816737192, 0.28093304749462866, 0.5

In [333]:
np.array_equal(data,d)

True

In [387]:
imgs = np.random.rand(1,1,12,12)

kernel_size = 3
stride = 2
padding = 0

con = conv2d(kernel_size, stride, padding)
data = con.img_to_conv2d(imgs)
print(data.shape)
for i in range(data.shape[0]):
    print(data[i].tolist())

Looking at window  0
ORIGIN:  0 0 COORD: 0 0
ORIGIN:  0 0 COORD: 0 1
ORIGIN:  0 0 COORD: 0 2
ORIGIN:  0 0 COORD: 1 0
ORIGIN:  0 0 COORD: 1 1
ORIGIN:  0 0 COORD: 1 2
ORIGIN:  0 0 COORD: 2 0
ORIGIN:  0 0 COORD: 2 1
ORIGIN:  0 0 COORD: 2 2
Looking at window  1
ORIGIN:  0 2 COORD: 0 2
ORIGIN:  0 2 COORD: 0 3
ORIGIN:  0 2 COORD: 0 4
ORIGIN:  0 2 COORD: 1 2
ORIGIN:  0 2 COORD: 1 3
ORIGIN:  0 2 COORD: 1 4
ORIGIN:  0 2 COORD: 2 2
ORIGIN:  0 2 COORD: 2 3
ORIGIN:  0 2 COORD: 2 4
Looking at window  2
ORIGIN:  0 4 COORD: 0 4
ORIGIN:  0 4 COORD: 0 5
ORIGIN:  0 4 COORD: 0 6
ORIGIN:  0 4 COORD: 1 4
ORIGIN:  0 4 COORD: 1 5
ORIGIN:  0 4 COORD: 1 6
ORIGIN:  0 4 COORD: 2 4
ORIGIN:  0 4 COORD: 2 5
ORIGIN:  0 4 COORD: 2 6
Looking at window  3
ORIGIN:  0 6 COORD: 0 6
ORIGIN:  0 6 COORD: 0 7
ORIGIN:  0 6 COORD: 0 8
ORIGIN:  0 6 COORD: 1 6
ORIGIN:  0 6 COORD: 1 7
ORIGIN:  0 6 COORD: 1 8
ORIGIN:  0 6 COORD: 2 6
ORIGIN:  0 6 COORD: 2 7
ORIGIN:  0 6 COORD: 2 8
Looking at window  4
ORIGIN:  0 8 COORD: 0 8
ORIGIN:

In [388]:
b_0_0 = np.array(imgs[0][0][0:3,0:3]).reshape(1,1,3,3)
b_0_1 = np.array(imgs[0][0][0:3,3:6]).reshape(1,1,3,3)
b_0_2 = np.array(imgs[0][0][0:3,6:9]).reshape(1,1,3,3)
b_0_3 = np.array(imgs[0][0][0:3,9:12]).reshape(1,1,3,3)

b_1_0 = np.array(imgs[0][0][3:6,0:3]).reshape(1,1,3,3)
b_1_1 = np.array(imgs[0][0][3:6,3:6]).reshape(1,1,3,3)
b_1_2 = np.array(imgs[0][0][3:6,6:9]).reshape(1,1,3,3)
b_1_3 = np.array(imgs[0][0][3:6,9:12]).reshape(1,1,3,3)

b_2_0 = np.array(imgs[0][0][6:9,0:3]).reshape(1,1,3,3)
b_2_1 = np.array(imgs[0][0][6:9,3:6]).reshape(1,1,3,3)
b_2_2 = np.array(imgs[0][0][6:9,6:9]).reshape(1,1,3,3)
b_2_3 = np.array(imgs[0][0][6:9,9:12]).reshape(1,1,3,3)

b_3_0 = np.array(imgs[0][0][9:12,0:3]).reshape(1,1,3,3)
b_3_1 = np.array(imgs[0][0][9:12,3:6]).reshape(1,1,3,3)
b_3_2 = np.array(imgs[0][0][9:12,6:9]).reshape(1,1,3,3)
b_3_3 = np.array(imgs[0][0][9:12,9:12]).reshape(1,1,3,3)

In [389]:
con = conv2d(kernel_size, stride, padding)

data00 = con.img_block_to_conv2d(b_0_0, 12, 12, 0, 0, 3, 3)
data01 = con.img_block_to_conv2d(b_0_1, 12, 12, 0, 1, 3, 3)
data02 = con.img_block_to_conv2d(b_0_2, 12, 12, 0, 2, 3, 3)
data03 = con.img_block_to_conv2d(b_0_3, 12, 12, 0, 3, 3, 3)

data10 = con.img_block_to_conv2d(b_1_0, 12, 12, 1, 0, 3, 3)
data11 = con.img_block_to_conv2d(b_1_1, 12, 12, 1, 1, 3, 3)
data12 = con.img_block_to_conv2d(b_1_2, 12, 12, 1, 2, 3, 3)
data13 = con.img_block_to_conv2d(b_1_3, 12, 12, 1, 3, 3, 3)

data20 = con.img_block_to_conv2d(b_2_0, 12, 12, 2, 0, 3, 3)
data21 = con.img_block_to_conv2d(b_2_1, 12, 12, 2, 1, 3, 3)
data22 = con.img_block_to_conv2d(b_2_2, 12, 12, 2, 2, 3, 3)
data23 = con.img_block_to_conv2d(b_2_3, 12, 12, 2, 3, 3, 3)

data30 = con.img_block_to_conv2d(b_3_0, 12, 12, 3, 0, 3, 3)
data31 = con.img_block_to_conv2d(b_3_1, 12, 12, 3, 1, 3, 3)
data32 = con.img_block_to_conv2d(b_3_2, 12, 12, 3, 2, 3, 3)
data33 = con.img_block_to_conv2d(b_3_3, 12, 12, 3, 3, 3, 3)

Virtual matrix slides  2 2
Starting window  0 0
Looking at window  0
ORIGIN:  0 0 COORD: 0 0 TCOORD:  0 0
ORIGIN:  0 0 COORD: 0 1 TCOORD:  0 1
ORIGIN:  0 0 COORD: 0 2 TCOORD:  0 2
ORIGIN:  0 0 COORD: 1 0 TCOORD:  1 0
ORIGIN:  0 0 COORD: 1 1 TCOORD:  1 1
ORIGIN:  0 0 COORD: 1 2 TCOORD:  1 2
ORIGIN:  0 0 COORD: 2 0 TCOORD:  2 0
ORIGIN:  0 0 COORD: 2 1 TCOORD:  2 1
ORIGIN:  0 0 COORD: 2 2 TCOORD:  2 2
Looking at window  1
ORIGIN:  0 2 COORD: 0 2 TCOORD:  0 2
ORIGIN:  0 2 COORD: 0 3 TCOORD:  0 3
ORIGIN:  0 2 COORD: 0 4 TCOORD:  0 4
ORIGIN:  0 2 COORD: 1 2 TCOORD:  1 2
ORIGIN:  0 2 COORD: 1 3 TCOORD:  1 3
ORIGIN:  0 2 COORD: 1 4 TCOORD:  1 4
ORIGIN:  0 2 COORD: 2 2 TCOORD:  2 2
ORIGIN:  0 2 COORD: 2 3 TCOORD:  2 3
ORIGIN:  0 2 COORD: 2 4 TCOORD:  2 4
Looking at window  5
ORIGIN:  2 0 COORD: 2 0 TCOORD:  2 0
ORIGIN:  2 0 COORD: 2 1 TCOORD:  2 1
ORIGIN:  2 0 COORD: 2 2 TCOORD:  2 2
ORIGIN:  2 0 COORD: 3 0 TCOORD:  3 0
ORIGIN:  2 0 COORD: 3 1 TCOORD:  3 1
ORIGIN:  2 0 COORD: 3 2 TCOORD:  3 2
O

In [390]:
d = con.aggregate([data00, data01, data02, data03, data10, data11, data12, data13, data20, data21, data22, data23, data30, data31, data32, data33])
for i in range(d.shape[0]):
    print(d[i].tolist())

[0.35740473385413785, 0.585799545063717, 0.574332255458146, 0.2690620861180377, 0.33739004282567764, 0.882079000615743, 0.9416195266103147, 0.7179799120266717, 0.3320995155999624]
[0.574332255458146, 0.8269408055828249, 0.07634053943222363, 0.882079000615743, 0.4317280251919986, 0.6429607485873633, 0.3320995155999624, 0.12058560615124925, 0.01570113142799079]
[0.07634053943222363, 0.397551077290787, 0.5904546954263464, 0.6429607485873633, 0.7410776802464104, 0.5847259270777297, 0.01570113142799079, 0.24733137257660542, 0.5094643172174828]
[0.5904546954263464, 0.9062164275470139, 0.09184722862452721, 0.5847259270777297, 0.5936385886178605, 0.9329897030117495, 0.5094643172174828, 0.8454868481740989, 0.33966000430909227]
[0.09184722862452721, 0.9018208027650778, 0.14885967600168581, 0.9329897030117495, 0.18821831187514393, 0.5043509616612053, 0.33966000430909227, 0.030464405385532856, 0.6907009280027834]
[0.9416195266103147, 0.7179799120266717, 0.3320995155999624, 0.49075380831364335, 0.5

In [391]:
for i in range(data.shape[0]):
    print(data[i].tolist())

[0.35740473385413785, 0.585799545063717, 0.574332255458146, 0.2690620861180377, 0.33739004282567764, 0.882079000615743, 0.9416195266103147, 0.7179799120266717, 0.3320995155999624]
[0.574332255458146, 0.8269408055828249, 0.07634053943222363, 0.882079000615743, 0.4317280251919986, 0.6429607485873633, 0.3320995155999624, 0.12058560615124925, 0.01570113142799079]
[0.07634053943222363, 0.397551077290787, 0.5904546954263464, 0.6429607485873633, 0.7410776802464104, 0.5847259270777297, 0.01570113142799079, 0.24733137257660542, 0.5094643172174828]
[0.5904546954263464, 0.9062164275470139, 0.09184722862452721, 0.5847259270777297, 0.5936385886178605, 0.9329897030117495, 0.5094643172174828, 0.8454868481740989, 0.33966000430909227]
[0.09184722862452721, 0.9018208027650778, 0.14885967600168581, 0.9329897030117495, 0.18821831187514393, 0.5043509616612053, 0.33966000430909227, 0.030464405385532856, 0.6907009280027834]
[0.9416195266103147, 0.7179799120266717, 0.3320995155999624, 0.49075380831364335, 0.5

In [392]:
np.array_equal(d, data)

True